In [6]:
pip install -r requirements.txt

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from collections import Counter
import numpy as np
import spacy
import nl_core_news_sm
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import random
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
import re
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from imblearn.over_sampling import SMOTE

ModuleNotFoundError: No module named 'pandas'

In [2]:
#read in data 
data_1= pd.read_excel("./Heijmans data final.xlsx")
#data_1.head()

In [3]:
#re-naming columns
data_1= data_1.rename(columns ={'Nr.': 'ID_num', "Beschrijving incident": "Incident_Description", "Subcategorie ongeval": "Sub_Categories", "Categorie ongeval": "Main_Categories", "Subcategorie(2) ongeval": "Reason_Categories", 
                                "Datum incident": "Incident_Date" })


#selecting the columns to work with
ID_num = data_1["ID_num"]
Incident_Description = data_1["Incident_Description"]
Sub_Categories = data_1["Sub_Categories"]
Reason_Categories= data_1["Reason_Categories"]
Main_Categories= data_1["Main_Categories"]
Incident_Date= data_1["Incident_Date" ]

#creating a new dataframe
c1= pd.Series(ID_num, name = "ID_num")
c2= pd.Series(Incident_Description, name = "Incident_Description")
c3= pd.Series(Sub_Categories, name= "Sub_Categories")
c4= pd.Series(Reason_Categories ,name= "Reason_Categories")
c5= pd.Series(Main_Categories, name= "Main_Categories")
c6= pd.Series(Incident_Date, name= "Incident_Date")

df = pd.concat([c1,c2,c3,c4,c5,c6], axis=1)
print(df.shape)

# EDA- Exploratory Data Analysis

#number of entries for each column
column_count= df.count()
print ("Column Count")
print(column_count)
print("")
print("")

#number of null/NAN entries 
print("Null Count ") 
print(df.isnull().sum())
print("")
print("")

#Count of uniques entries
a= df["Main_Categories"].nunique()
print("Unique_main_categories:", a)
c= df["Sub_Categories"].nunique()
print("Unique_subcategories:", c)
e= df["Reason_Categories"].nunique()
print("Unique_reasons:", e)
print("")
print("")

# class distribution Percentages 
class_dist= (df.groupby("Main_Categories").size()/df["Main_Categories"].count())*100
print(class_dist)

# removing nan
df["Main_Categories"].replace(r'^\s+$', np.nan, regex=True)
mc_df_1=df.dropna(subset=["Main_Categories"])

print("Count of new data frame ")
print(mc_df_1.count())
print("")
print("")
print("Double Checking for NaN")
print(mc_df_1.isnull().sum())

(18516, 6)
Column Count
ID_num                  18516
Incident_Description    17909
Sub_Categories           3276
Reason_Categories        2340
Main_Categories          3307
Incident_Date           18516
dtype: int64


Null Count 
ID_num                      0
Incident_Description      607
Sub_Categories          15240
Reason_Categories       16176
Main_Categories         15209
Incident_Date               0
dtype: int64


Unique_main_categories: 10
Unique_subcategories: 43
Unique_reasons: 154


Main_Categories
Aanrijding                                      5.624433
Agressie en extreme lichamelijke inspanning     2.056244
Andere vormen van contact met voorwerpen       14.454188
BHV inzet op locatie                            3.386755
Gevaarlijke atmosfeer                           0.302389
Gevaarlijke stoffen, brand en explosies         3.144844
Gezondheidsschade                               0.514061
Machines en handgereedschap                    12.760810
Vallen (van personen of voor

In [4]:
# removing noise from the data 
mc_df_1=mc_df_1[Main_Categories!="overig"]  # other category adds no value to data and reduces model's performance
mc_df_2= mc_df_1[Main_Categories!="Gevaarlijke atmosfeer"] # too few entries, the machine learns nothing
mc_df= mc_df_2[Main_Categories!="Gezondheidsschade"] # too few entries, the machine learns nothing

C:\Users\byri\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\byri\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\byri\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


******* PREPROCESSING ********

In [5]:
##creating stop words list, and lemmatizer  

stopword_list = set(nltk.corpus.stopwords.words('dutch'))
#add words that aren't in the NLTK stopwords list
added_stopwords = ['+', '1e', '2e', '<', '>', 'a', 'a1', 'a12', 'a2', 'a9',"één", "gis", "app", "h", 'o.a.', 'oa', 'en/of']
updated_stopword_list = stopword_list.union(added_stopwords)
#print(updated_stopword_list)

# Dutch spacy tokens 
nlp = nl_core_news_sm.load()

def custom_lemmatizer(data):
    return [(token.lemma_) for token in nlp(data) if not token.is_digit and not token.is_punct and not token.is_space]



********MODEL BUILDING********

In [9]:
# split the dataset into training and validation datasets 
text = mc_df["Incident_Description"].astype(str)
label = mc_df["Main_Categories"]
# smote= SMOTE("auto")
#label, text = smote.fit_sample(label,text)
text_train, text_test, label_train, label_test, = model_selection.train_test_split(text,label, test_size = 0.2)


In [10]:
## 𝑇𝐹×𝐼𝐷𝐹
# word level tf-idf
tfidf_vectorizer= TfidfVectorizer(max_features=12000,  stop_words=updated_stopword_list, max_df= 0.5, tokenizer=custom_lemmatizer)


In [12]:
#fit & transform for cross validation 
train_tfidf = tfidf_vectorizer.fit(text)
tfidf_text_v= train_tfidf.transform(text)
# smote= SMOTE("auto")
# tfidf_text_v_train, text = smote.fit_sample(label,text)

C:\Users\byri\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [13]:
# fit and transform for test  & train
tfidf_text_v_train= train_tfidf.transform(text_train)
tfidf_text_v_test= train_tfidf.transform(text_test)


In [74]:
#SVM
SVM_cl= LinearSVC()
SVM_cl.fit(tfidf_text_v_train, label_train)
print ("SVM Score:", SVM_cl.score(tfidf_text_v_test, label_test))

SVM Score: 0.7395833333333334


In [66]:
#SVM with L1 regularization 
#learning_rate=1, max_features=2, max_depth=2,
SVM_cl_p= LinearSVC(penalty="l1", loss='squared_hinge',dual=False )
SVM_cl_p.fit(tfidf_text_v_train, label_train)
#print ("SVM_P Score:", SVM_cl_p.score(tfidf_text_v_test, label_test))
print("SVM_P score (test):",SVM_cl_p.score(tfidf_text_v_test, label_test))
print("SVM_P  score (training):", SVM_cl_p.score(tfidf_text_v_train, label_train))

SVM_P score (test): 0.7291666666666666
SVM_P  score (training): 0.928695652173913


In [21]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
model_params= {"svm": {"model":svm.SVC(gamma="auto"), "params": {"C": [1,10,20], "kernel":["rbf", "linear"], "penalty":["l1", "L2"], "dual": ["False", "True"], "loss":'squared_hinge' }}, 
              "decision_tree": {"model":DecisionTreeClassifier(),"params": {"n_estimators": [1,10,20], "criterion":['gini','entropy']}}, 
               "naive_bayes":{"model":MultinomialNB(),"params": {"C": [1,10,20]}},
               "kNN":{"model":KNeighborsClassifier(),"params": { "n_neighbors":[3,5,10], "weights":["uniform", "distance"], "algorithm":['ball_tree','kd_tree'] }},
               "xgb_cl":{"model":xgb.XGBClassifier(random_state=4),"params": { "n_estimators": [1,50,10,100], "learning_rate":[0.1,0.5,1.0,2.0] }}}


In [22]:
scores=[]

for model_name, mp in model_params.items():
    clf= GridSearchCV(mp["model"], mp ["params"], cv=5, return_train_score=False )
clf.fit(tfidf_text_v_train, label_train)
scores.append({"model": model_name, "best_score":clf.best_score_, "best_params": clf.best_params_})

# classifiers.append(
#     clf.grid_search_dummy_classifier(dict(
#         strategy=['most_frequent','stratified','uniform'])))
# df_gridS=pd.DataFrame(clf.cv_results_)
# df_gridS[['param_C', 'param_kernel', 'mean_test_score','rank_test_score']]

In [27]:
imbalanced, imbalanced_classifiers = clf.fit_predict_measure(
    'Imbalanced', tfidf_text_v_train, tfidf_text_v_test, label_train,label_test, list(labels[0]), scores)
imbalanced



AttributeError: 'GridSearchCV' object has no attribute 'fit_predict_measure'

In [26]:
clf.cv_results_
df_gridS=pd.DataFrame(clf.cv_results_)
df_gridS[['param_C', 'param_kernel', 'mean_test_score','rank_test_score']]

KeyError: "['param_kernel', 'param_C'] not in index"

In [23]:
print(scores)

[{'model': 'xgb_cl', 'best_score': 0.6752173913043478, 'best_params': {'learning_rate': 0.5, 'n_estimators': 100}}]


In [120]:
model_params= {"svm": {"model":svm.SVC(gamma="auto"), "params": {"kernel":["rbf", "linear"]}}, 
              "decision_tree": {"model":DecisionTreeClassifier(), "params": {"n_estimators": [1,10,20]}}, 
               "naive_bayes":{"model":MultinomialNB(),"params": {"params": {"C": [1,10,20]}}},
               "kNN":{"model":KNeighborsClassifier(),"params": { "n_neighbors":[5] }}}


In [121]:
scores=[]

for model_name, mp in model_params.items():
    clf= GridSearchCV(mp["model"], mp ["params"], cv=5, return_train_score=False )
clf.fit(tfidf_text_v_train, label_train)
scores.append({"model": model_name, "best_score":clf.best_score_, "best_params": clf.best_params_})
print(scores)

[{'model': 'kNN', 'best_score': 0.6199999999999999, 'best_params': {'n_neighbors': 5}}]


In [76]:

svc = svm.SVC()
clf = GridSearchCV(svm.SVC(gamma="auto"), {'C': [1,10,20], 'kernel': ['linear', 'rbf']},cv=5)
clf.fit(tfidf_text_v_train, label_train)
clf.cv_results_

{'mean_fit_time': array([0.69477353, 0.33213081, 0.74147754, 0.49249501, 0.71868982,
        0.53139067]),
 'std_fit_time': array([0.03860629, 0.00403855, 0.02723302, 0.03015224, 0.02241706,
        0.00831969]),
 'mean_score_time': array([0.07998576, 0.07877822, 0.08059487, 0.09414763, 0.07946672,
        0.08856382]),
 'std_score_time': array([0.00159548, 0.00141426, 0.00158695, 0.01088174, 0.00107235,
        0.00146574]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['linear', 'rbf', 'linear', 'rbf', 'linear', 'rbf'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'linear'},
  {'C': 1, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'}],


In [82]:
df_gridS=pd.DataFrame(clf.cv_results_)
df_gridS[['param_C', 'param_kernel', 'mean_test_score','rank_test_score']]

,param_C,param_kernel,mean_test_score,rank_test_score
0,1,linear,0.709130,1
1,1,rbf,0.523478,4
2,10,linear,0.698696,2
3,10,rbf,0.523478,4
4,20,linear,0.696522,3
5,20,rbf,0.523478,4


In [11]:
labels_predict_SVM= SVM_cl.predict(tfidf_text_v_test)
print(classification_report(label_test, labels_predict_SVM))


                                             precision    recall  f1-score   support

                                 Aanrijding       0.73      0.69      0.71        32
Agressie en extreme lichamelijke inspanning       1.00      0.18      0.30        17
   Andere vormen van contact met voorwerpen       0.55      0.47      0.51        94
                       BHV inzet op locatie       0.67      0.53      0.59        19
    Gevaarlijke stoffen, brand en explosies       0.86      0.29      0.43        21
                Machines en handgereedschap       0.66      0.58      0.62        86
        Vallen (van personen of voorwerpen)       0.76      0.91      0.83       307

                                   accuracy                           0.72       576
                                  macro avg       0.75      0.52      0.57       576
                               weighted avg       0.72      0.72      0.70       576



In [12]:
print(confusion_matrix(label_test, labels_predict_SVM))


[[ 22   0   0   0   0   1   9]
 [  1   3   3   0   0   0  10]
 [  0   0  44   2   0  17  31]
 [  0   0   0  10   0   0   9]
 [  1   0   3   0   6   1  10]
 [  3   0  13   1   1  50  18]
 [  3   0  17   2   0   7 278]]


In [70]:
results= cross_val_score(SVM_cl, tfidf_text_v, label, cv = 5)
print(results)
print("mean=",results.mean())

[0.71527778 0.68       0.71304348 0.68695652 0.66434783]
mean= 0.6919251207729469


In [50]:
# multi-class One-vs-One scheme compares every unique pairwise combination of classes
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True))
y_score = classifier.fit(tfidf_text_v_train, label_train).decision_function(tfidf_text_v_test)


y_prob = classifier.predict_proba(tfidf_text_v_test)

macro_roc_auc_ovo = roc_auc_score( label_test, y_prob, multi_class="ovo",
                                  average="macro")
macro_roc_auc_ovr = roc_auc_score(label_test, y_prob, multi_class="ovr",
                                  average="macro")

In [18]:
print ("One-vs-One ROC AUC scores:", macro_roc_auc_ovo)
print("One-vs-Rest ROC AUC scores:" ,macro_roc_auc_ovr)

One-vs-One ROC AUC scores: 0.8472881488048246
One-vs-Rest ROC AUC scores: 0.8625495688325764


In [19]:
#KNN
KNN_cl = KNeighborsClassifier()
KNN_cl.fit(tfidf_text_v_train, label_train)
print ("KNN Score:", KNN_cl.score(tfidf_text_v_test, label_test))

KNN Score: 0.65625


In [20]:
labels_predict_KNN= KNN_cl.predict(tfidf_text_v_test)
print(classification_report(label_test, labels_predict_KNN))

                                             precision    recall  f1-score   support

                                 Aanrijding       0.89      0.53      0.67        32
Agressie en extreme lichamelijke inspanning       0.00      0.00      0.00        17
   Andere vormen van contact met voorwerpen       0.57      0.29      0.38        94
                       BHV inzet op locatie       0.62      0.53      0.57        19
    Gevaarlijke stoffen, brand en explosies       0.67      0.19      0.30        21
                Machines en handgereedschap       0.48      0.47      0.47        86
        Vallen (van personen of voorwerpen)       0.69      0.91      0.79       307

                                   accuracy                           0.66       576
                                  macro avg       0.56      0.42      0.45       576
                               weighted avg       0.63      0.66      0.62       576



In [21]:
print(confusion_matrix(label_test, labels_predict_KNN))

[[ 17   0   0   1   0   4  10]
 [  2   0   1   1   0   1  12]
 [  0   1  27   1   0  21  44]
 [  0   0   0  10   1   0   8]
 [  0   0   3   0   4   3  11]
 [  0   0   7   1   0  40  38]
 [  0   1   9   2   1  14 280]]


In [15]:
cross_val_score(KNN_cl, tfidf_text_v, label, cv = 5)

array([0.55034722, 0.56347826, 0.54086957, 0.61217391, 0.53043478])

In [22]:
#DT
DT_cl = DecisionTreeClassifier()
DT_cl.fit(tfidf_text_v_train, label_train)
print ("DT Score:", DT_cl.score(tfidf_text_v_test, label_test))

DT Score: 0.6319444444444444


In [23]:
labels_predict_DT= DT_cl.predict(tfidf_text_v_test)
print(classification_report(label_test, labels_predict_DT))

                                             precision    recall  f1-score   support

                                 Aanrijding       0.46      0.53      0.49        32
Agressie en extreme lichamelijke inspanning       0.12      0.06      0.08        17
   Andere vormen van contact met voorwerpen       0.46      0.41      0.44        94
                       BHV inzet op locatie       0.56      0.47      0.51        19
    Gevaarlijke stoffen, brand en explosies       0.40      0.19      0.26        21
                Machines en handgereedschap       0.53      0.44      0.48        86
        Vallen (van personen of voorwerpen)       0.73      0.83      0.78       307

                                   accuracy                           0.63       576
                                  macro avg       0.47      0.42      0.43       576
                               weighted avg       0.61      0.63      0.62       576



In [24]:
print(confusion_matrix(label_test, labels_predict_DT))

[[ 17   2   2   0   1   2   8]
 [  1   1   4   1   0   0  10]
 [  1   1  39   1   1  15  36]
 [  1   0   0   9   0   0   9]
 [  3   0   3   0   4   2   9]
 [  3   1  19   1   3  38  21]
 [ 11   3  17   4   1  15 256]]


In [25]:
cross_val_score(DT_cl, tfidf_text_v, label, cv = 5)

array([0.61284722, 0.61565217, 0.60695652, 0.56173913, 0.51478261])

In [113]:
xgb_cl=xgb.XGBClassifier(random_state=4)
xgb_cl.fit(tfidf_text_v_train, label_train)
xgb_cl.score(tfidf_text_v_test, label_test)
print("Accuracy score (test):",xgb_cl.score(tfidf_text_v_test, label_test))
print("Accuracy score (training):", xgb_cl.score(tfidf_text_v_train, label_train))


0.6979166666666666

In [115]:
cross_val_score(xgb_cl, tfidf_text_v, label, cv = 5)

array([0.68055556, 0.68      , 0.66608696, 0.69391304, 0.60695652])

In [26]:
# NB
NB_cl= MultinomialNB()
NB_cl.fit(tfidf_text_v_train, label_train)

print ("NB Score:", NB_cl.score(tfidf_text_v_test, label_test))

NB Score: 0.5885416666666666


In [27]:
labels_predict_NB= NB_cl.predict(tfidf_text_v_test)
print(classification_report(label_test, labels_predict_NB))

                                             precision    recall  f1-score   support

                                 Aanrijding       0.00      0.00      0.00        32
Agressie en extreme lichamelijke inspanning       0.00      0.00      0.00        17
   Andere vormen van contact met voorwerpen       0.91      0.11      0.19        94
                       BHV inzet op locatie       1.00      0.26      0.42        19
    Gevaarlijke stoffen, brand en explosies       0.00      0.00      0.00        21
                Machines en handgereedschap       0.77      0.20      0.31        86
        Vallen (van personen of voorwerpen)       0.57      1.00      0.73       307

                                   accuracy                           0.59       576
                                  macro avg       0.46      0.22      0.24       576
                               weighted avg       0.60      0.59      0.48       576



C:\Users\byri\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
print(confusion_matrix(label_test, labels_predict_NB))

[[  0   0   0   0   0   0  32]
 [  0   0   0   0   0   0  17]
 [  0   0  10   0   0   5  79]
 [  0   0   0   5   0   0  14]
 [  0   0   1   0   0   0  20]
 [  0   0   0   0   0  17  69]
 [  0   0   0   0   0   0 307]]


In [29]:
cross_val_score(NB_cl, tfidf_text_v, label, cv = 5)


array([0.56597222, 0.55652174, 0.55826087, 0.57391304, 0.57217391])

0.6102125603864733


In [41]:

NB_cl= MultinomialNB(Laplace= 0.5)
NB_cl.fit(tfidf_text_v_train, label_train).predict(text_test)
#y_pred = gnb.fit(X_train, y_train).predict(X_test)

print ("NB Score:", NB_cl.score(tfidf_text_v_test, label_test))

TypeError: __init__() got an unexpected keyword argument 'laplace'

In [36]:
# multi-class One-vs-One scheme compares every unique pairwise combination of classes
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True))
y_score = classifier.fit(tfidf_text_v_train, label_train).decision_function(tfidf_text_v_test)


y_prob = classifier.predict_proba(tfidf_text_v_test)

macro_roc_auc_ovo = roc_auc_score( label_test, y_prob, multi_class="ovo",
                                  average="macro")
weighted_roc_auc_ovo = roc_auc_score(label_test, y_prob, multi_class="ovo",
                                     average="weighted")
macro_roc_auc_ovr = roc_auc_score(label_test, y_prob, multi_class="ovr",
                                  average="macro")
weighted_roc_auc_ovr = roc_auc_score(label_test, y_prob, multi_class="ovr",
                                     average="weighted")
print("One-vs-One ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
      "(weighted by prevalence)"
      .format(macro_roc_auc_ovo, weighted_roc_auc_ovo))
print("One-vs-Rest ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
      "(weighted by prevalence)"
      .format(macro_roc_auc_ovr, weighted_roc_auc_ovr))

One-vs-One ROC AUC scores:
0.502078 (macro),
0.499855 (weighted by prevalence)
One-vs-Rest ROC AUC scores:
0.503402 (macro),
0.498358 (weighted by prevalence)


In [41]:
df["Incident_Date" ].sort_values

<bound method Series.sort_values of 0       2019-12-10
1       2019-11-22
2       2019-11-18
3       2019-11-18
4       2019-11-15
           ...    
18511   2007-04-20
18512   2007-04-02
18513   2006-12-11
18514   2006-10-23
18515   2006-09-11
Name: Incident_Date, Length: 18516, dtype: datetime64[ns]>

In [13]:
#Divide dataset equally- in 6.5 years
df["Incident_Date"] = pd.to_datetime(df["Incident_Date"], format='%Y%m%d')
df = df.set_index("Incident_Date")
df = df.sort_index()
train = df['2006-09-11':'2013-05-10']
test  = df['2013-06-11':]
print('Train Dataset:',train.shape)
print('Test Dataset:',test.shape)


Train Dataset: (1292, 5)
Test Dataset: (17147, 5)


In [19]:
#Divide dataset 80:20 
df["Incident_Date"] = pd.to_datetime(df["Incident_Date"], format='%Y%m%d')
df = df.set_index("Incident_Date")
df = df.sort_index()
train = df['2006-09-11':'2016-01-10']
test  = df['2016-01-11':]
print('Train Dataset:',train.shape)
print('Test Dataset:',test.shape)


Train Dataset: (4039, 5)
Test Dataset: (14477, 5)


In [33]:
#Divide dataset 90:10 
mc_df["Incident_Date"] = pd.to_datetime(mc_df["Incident_Date"], format='%Y%m%d')
mc_df = mc_df.set_index("Incident_Date")
mc_df = mc_df.sort_index()
df_ey = mc_df['2006-09-11':'2017-04-10']
df_ly  = mc_df['2017-04-11':]
print('Train Dataset:',train.shape)
print('Test Dataset:',test.shape)


Train Dataset: (7585, 5)
Test Dataset: (10931, 5)


C:\Users\byri\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [40]:
# split the dataset into training and validation datasets 
text_1 = df_ey["Incident_Description"].astype(str)
label_1 = df_ey["Main_Categories"]
text_1_train, text_1_test, label_1_train, label_1_test, = model_selection.train_test_split(text_1,label_1, test_size = 0.2)



In [41]:
train_tfidf = tfidf_vectorizer.fit(text_1)
tfidf_text_v= train_tfidf.transform(text_1)

C:\Users\byri\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [42]:
# fit and transform for test  & train
tfidf_text_v_train= train_tfidf.transform(text_1_train)
tfidf_text_v_test= train_tfidf.transform(text_1_test)


In [52]:
SVM_cl_p= LinearSVC(penalty="l1", loss='squared_hinge',dual=False )
SVM_cl_p.fit(tfidf_text_v_train, label_1_train)
#print ("SVM_P Score:", SVM_cl_p.score(tfidf_text_v_test, label_test))
print("SVM_P score (test):",SVM_cl_p.score(tfidf_text_v_test, label_1_test))
print("SVM_P  score (training):", SVM_cl_p.score(tfidf_text_v_train, label_1_train))

SVM_P score (test): 0.6548223350253807
SVM_P  score (training): 0.9224904701397713


In [46]:
classes = list(np.unique(df_ly.Main_Categories))
for i in range 0,df_ly["Incident_Description"].size
prob=SVM_cl_p.predict()
index=np.argmax(prob[0])
# results["text_1"].append(test_feature_label_frame["path"][i])
# results["label_1"].append(classes[index])

TypeError: predict() missing 1 required positional argument: 'X'

In [49]:
df_ly["Incident_Description"][0].shape

AttributeError: 'str' object has no attribute 'shape'

In [51]:
print(text_1_train.shape)

(787,)
